In [1]:
! pip install requests pandas gspread

In [2]:
from datetime import date, timedelta
import numpy as np
import pandas as pd
from data_project.tapdb import TapDBClient
from data_project.gsheets import DateSheet
from config import TAPDB_USER, TAPDB_PASS
import asyncio

target_date = (date.today() - timedelta(days=1)).isoformat()
#target_date = '2024-07-27'

if isinstance(target_date, str):
    target_date = date.fromisoformat(target_date)

async def create_tapdb():
    return TapDBClient(TAPDB_USER, TAPDB_PASS, target_date=target_date)

async def create_sheet():
    return DateSheet('[RP_CN] Revenue Report', date=target_date.isoformat())

task_tapdb = asyncio.create_task(create_tapdb())
task_sheet = asyncio.create_task(create_sheet())

await task_tapdb
await task_sheet

tapdb = task_tapdb.result()
sheet = task_sheet.result()

In [9]:
# [RP_CN] Revenue Report >> 活躍設備
sheet.change_sheet('Daily Re.')
data = tapdb.get_data('RP CN (PRO)', 'active', dim='os', device=True)

sheet.update_cells('活躍設備\n(tap db)', int(data['DAU_DEVICE'].sum()))
sheet.update_cells('活躍設備\n(android)', int(data.loc['Android', 'DAU_DEVICE']))
data

,key,DAU_DEVICE,newDAU_DEVICE,DAU_DEVICE2,DAU_DEVICE3,DAU_DEVICE7,DAU_DEVICE14,DAU_DEVICE30
key,,,,,,,,
Android,Android,36912,5038,31874,30202,26453,23142,19122
iOS,iOS,4601,1039,3562,3223,134,74,13


In [4]:
# [RP_CN] Revenue Report >> 新增設備
sheet.change_sheet('Daily Re.')
data = tapdb.get_data('RP CN (PRO)', 'source', dim='os', device=True)
sheet.update_cells('新增設備Total\n(Tap DB)', int(data['newDevices'].sum()))
sheet.update_cells('AND新設備\n(Tap DB)', int(data.loc['Android', 'newDevices']))
data


,key,newPlayers,totalCharge,totalChargeAmount,firstChargePlayers,firstChargeAmount,newDevices,convertedDevices,devicesConvertRate,dayRetainedRate1,dayRetainedRate7,chargeRate,firstChargeRate,ARPU,ARPPU
key,,,,,,,,,,,,,,,
Android,Android,5550,52,542.0,33,175,5038,4277,84.89,None,None,1.03,0.66,0.03,5.30
iOS,iOS,857,15,455.9,9,411,1039,865,83.25,None,None,1.44,0.87,0.40,45.67


In [10]:
# [RP_CN] Revenue Report >> 轉化率
data = tapdb.get_data('RP CN (PRO)', 'source', dim='ver',pf='and')
row = data.loc[max(data.index)]

sheet.change_sheet('轉化率')
sheet.update_cells('版本', row['key'])
sheet.update_cells('新增設備', row['newDevices'])
sheet.update_cells('轉化設備', row['convertedDevices'])

data

,key,newPlayers,totalCharge,totalChargeAmount,firstChargePlayers,firstChargeAmount,newDevices,convertedDevices,devicesConvertRate,dayRetainedRate1,dayRetainedRate7,chargeRate,firstChargeRate,ARPU,ARPPU
key,,,,,,,,,,,,,,,
9.0.0.377,9.0.0.377,5550,52,542,33,175,4503,4003,88.90,None,None,1.15,0.73,0.04,5.3
8.0.0.353,8.0.0.353,0,0,0,0,0,201,77,38.31,None,None,0.00,0.00,0.00,0.0
1.0.0.146,1.0.0.146,0,0,0,0,0,148,100,67.57,None,None,0.00,0.00,0.00,0.0
7.0.0.325,7.0.0.325,0,0,0,0,0,144,75,52.08,None,None,0.00,0.00,0.00,0.0
6.0.0.274,6.0.0.274,0,0,0,0,0,27,17,62.96,None,None,0.00,0.00,0.00,0.0
7.0.0.308,7.0.0.308,0,0,0,0,0,6,4,66.67,None,None,0.00,0.00,0.00,0.0
8.0.0.352,8.0.0.352,0,0,0,0,0,3,0,0.00,None,None,0.00,0.00,0.00,0.0
5.0.0.256,5.0.0.256,0,0,0,0,0,2,0,0.00,None,None,0.00,0.00,0.00,0.0
5.0.0.239,5.0.0.239,0,0,0,0,0,1,1,100.00,None,None,0.00,0.00,0.00,0.0


In [6]:
# [RP_CN] Revenue Report >> 設備留存
data = tapdb.get_data('RP CN (PRO)', 'retain', begin_date='2024-04-01', pf='and', type='both')
data.sort_index(inplace=True)
data['key'] = data['key'].astype(str)
data_drr = data.loc[:, data.columns.str.contains('DRR')]
data = pd.concat([data.loc[:, ['newDevices']], data_drr / 100], axis=1)

data.to_csv(r'D:\Serena_Yu\Desktop\設備留存\retain.csv')
data

,newDevices,DRR1,DRR2,DRR3,DRR4,DRR5,DRR6,DRR7,DRR8,DRR9,...,DRR26,DRR27,DRR28,DRR29,DRR30,DRR60,DRR90,DRR120,DRR150,DRR180
key,,,,,,,,,,,,,,,,,,,,,
2024-04-01,3392,0.3420,0.2503,0.2114,0.1816,0.1689,0.1518,0.1456,0.1344,0.1203,...,0.0604,0.0596,0.0566,0.0522,0.0545,0.0239,0.0165,0.0091,NaN,NaN
2024-04-02,3796,0.3311,0.2323,0.1955,0.1752,0.1610,0.1454,0.1394,0.1262,0.1117,...,0.0537,0.0522,0.0548,0.0493,0.0469,0.0224,0.0124,NaN,NaN,NaN
2024-04-03,6546,0.3262,0.1975,0.1573,0.1257,0.1121,0.1059,0.0930,0.0866,0.0880,...,0.0409,0.0530,0.0527,0.0466,0.0393,0.0182,0.0124,NaN,NaN,NaN
2024-04-04,10555,0.2928,0.1751,0.1122,0.1022,0.0968,0.0853,0.0798,0.0794,0.0950,...,0.0534,0.0626,0.0474,0.0418,0.0394,0.0124,0.0099,NaN,NaN,NaN
2024-04-05,8652,0.2851,0.1472,0.1291,0.1153,0.1066,0.0993,0.1053,0.1192,0.0987,...,0.0624,0.0561,0.0520,0.0474,0.0452,0.0170,0.0116,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-25,4893,0.3513,0.2350,0.2007,0.1796,0.1404,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-26,4663,0.3365,0.2370,0.2044,0.1628,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-27,4797,0.3290,0.2266,0.1655,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# [RP_CN] Revenue Report >> Wkly Re.
sheet.change_sheet('Wkly Re.')
data = tapdb.get_data('RP CN (PRO)', 'wau', device=True)
target_week=target_date-timedelta(days=target_date.weekday())
row=data.loc[target_week]

sheet.update_cells('活躍設備\n(tap db)', int(row['WAU_DEVICE']))


data = tapdb.get_data('RP CN (PRO)', 'wau', device=True, pf='and')
target_week=target_date-timedelta(days=target_date.weekday())
row=data.loc[target_week]
sheet.update_cells('活躍設備\n(Android)', int(row['WAU_DEVICE']))

data

ValueError: Given date 2024-07-29 not found, min 日期 max 2024-08-11